# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import time
import torch
import numpy as np

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxend module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = False
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

Approxend module not found, cannot use gamepad
***ExpRun**: Loading pointer config file:
	C:\Users\lotzi\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	c:\Users\lotzi\Work\_Config\BerryPicker\cfg\settings.yaml
Using device: cuda


In [ ]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

Hostname is glassy
Path for external experiments: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
Path for external data: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Experiment config path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
***ExpRun**: Experiment data path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Experiment demonstration copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\robot_al5d
***ExpRun**: Experiment automate copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\automate
***ExpRun**: Experiment behavior_cloning copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\behavior_cloning
***Exp

In [4]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
run = "runbc_bc_lstm_0001"

exp = Config().get_experiment(experiment, run)

exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])
exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

['bc_0001',
 'bc_compare_00',
 'bc_lstm_00',
 'bc_lstm_0001',
 'bc_lstm_mdn_00',
 'bc_lstm_mdn_0001',
 'bc_lstm_residual_0001',
 'bc_lstm_resid_00',
 'bc_mlp_00',
 'bc_mlp_0001',
 'bc_run_00',
 'bc_run_0001',
 'bc_verify_lstm_00',
 'bc_verify_lstm_mdn_00',
 'bc_verify_lstm_resid_00',
 'bc_verify_mlp_00',
 'mdn_00',
 'mdn_for_bc_00',
 'runbc_bc_lstm_0001',
 'runbc_bc_lstm_mdn_0001',
 'runbc_bc_lstm_residual_0001',
 'runbc_bc_mlp_0001',
 '_defaults_behavior_cloning']
***ExpRun**: Configuration for exp/run: behavior_cloning/runbc_bc_lstm_0001 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_0001 successfully loaded
***Exp

In [5]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path))   

<All keys matched successfully>

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

FIXME: this controller loop needs to be made into a function and cleaned up. 

In [6]:
last_interval = 0
controller_interval = 0.1

transform = sp_helper.get_transform_to_sp(exp_sp)
# the backlog of the last sequence length steps
z_seq = []

while True:
    start_time = time.time() 
    # Let us assume that the first image here is the one
    camera_controller.update()
    image = camera_controller.images[exp["control_camera"]]
    # FIXME: this is totally won't work like this
    # there should be a notebook with the input processing???
    sensor_readings = sp_helper.load_capture_to_tensor(image, transform, device)
    z = sp.process(sensor_readings[0])
    print(z)
    if exp_bc["sequence_length"]: # no sequence processsing
        z_seq.append(z)
        if len(z_seq) > exp_bc["sequence_length"]:
            z_seq.pop(0)
        if len(z_seq) < exp_bc["sequence_length"]:
            ## we don't have enough in the queue, wait.
            print("Not acting, waiting on the queue", flush=True)
            pass
        else:
            # FIXME: this should be applied online, not step by step
            z_seq_np = np.array(z_seq)
            z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(device)
    else:
        z_tensor = torch.tensor(z).unsqueeze(0).to(device)
    if not model.stochastic:                
        a_pred = model(z_tensor)
    else:
        a_pred = model.forward_and_sample(z_tensor)

    print(a_pred, flush=True)

    if robot_controller:
        # move the robot
        pos_target = PositionController.from_normalized_vector(a_pred, exp_robot_controller)

        # ask the user whether we indeed want to continue moving?
        pos_current = robot_controller.get_position()
        distance = pos_current.empirical_distance(exp_robot_controller, pos_target)
        print(f"Next position: {pos_target} at distance={distance}")
        ans = input("Proceed with move? (Y/N/Q)")
        if ans.lower() == "y":
            robot_controller.move(pos_target)
        elif ans.lower() == "q":
            break # break out from the loop
    else:
        pass

    # ensuring that the execution happens 
    end_time = time.time() 
    execution_time = end_time - start_time 
    last_interval = execution_time
    time_to_sleep = max(0.0, controller_interval - execution_time) 
    time.sleep(time_to_sleep) 


Error at visualization? 


KeyError: 'dev0'

In [ ]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()